In [26]:
import sqlite3
import csv
import os

# Paths to your CSV files
COUNTRY_CSV   = '/Users/nicholasrichards/Desktop/QTM_350/final_project/API_NY.GDP.PCAP.KD_DS2_en_csv_v2_19405/Metadata_Country_API_NY.GDP.PCAP.KD_DS2_en_csv_v2_19405.csv'
INDICATOR_CSV = '/Users/nicholasrichards/Desktop/QTM_350/final_project/API_NY.GDP.PCAP.KD_DS2_en_csv_v2_19405/Metadata_Indicator_API_NY.GDP.PCAP.KD_DS2_en_csv_v2_19405.csv'
DATA_CSV      = '/Users/nicholasrichards/Desktop/QTM_350/final_project/API_NY.GDP.PCAP.KD_DS2_en_csv_v2_19405/API_NY.GDP.PCAP.KD_DS2_en_csv_v2_19405.csv'
DB_PATH       = 'gdp.db'

# Remove any existing DB to avoid lock issues
if os.path.exists(DB_PATH):
    try:
        os.remove(DB_PATH)
    except PermissionError:
        pass

# Helper to sanitize column names into valid SQLite identifiers
def sanitize(col_name):
    c = col_name.strip()
    for ch in [' ', '/', '-', '.', '(', ')', '%']:
        c = c.replace(ch, '_')
    if c and c[0].isdigit():
        c = '_' + c
    return c

# Connect to SQLite with a busy timeout
conn = sqlite3.connect(DB_PATH, timeout=30)
cur = conn.cursor()
cur.execute('PRAGMA busy_timeout = 30000;')

# 1. Load country metadata into staging
cur.execute('DROP TABLE IF EXISTS staging_country_meta;')
cur.execute('''
    CREATE TABLE staging_country_meta(
        CountryName TEXT,
        CountryCode TEXT,
        ExtraCol    TEXT
    );
''')
with open(COUNTRY_CSV, encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    for row in reader:
        if not row:
            continue
        cur.execute(
            'INSERT INTO staging_country_meta VALUES (?, ?, ?);',
            (row[0], row[1], row[2] if len(row) > 2 else None)
        )

# 2. Load indicator metadata into staging
cur.execute('DROP TABLE IF EXISTS staging_indicator_meta;')
cur.execute('''
    CREATE TABLE staging_indicator_meta(
        IndicatorName TEXT,
        IndicatorCode TEXT,
        Topic         TEXT,
        Source        TEXT,
        ExtraCol      TEXT
    );
''')
with open(INDICATOR_CSV, encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        if not row:
            continue
        cur.execute(
            'INSERT INTO staging_indicator_meta VALUES (?, ?, ?, ?, ?);',
            (row[0], row[1], row[2] if len(row) > 2 else None,
             row[3] if len(row) > 3 else None,
             row[4] if len(row) > 4 else None)
        )

# 3. Load GDP per capita CSV into staging_wdi_raw
cur.execute('DROP TABLE IF EXISTS staging_wdi_raw;')
with open(DATA_CSV, encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    # Skip preamble rows until header
    for row in reader:
        if row and row[0] == 'Country Name':
            headers = row
            break
    else:
        raise ValueError("Header row starting with 'Country Name' not found")

    hdr_clean = [sanitize(h) for h in headers]
    cols_ddl = ', '.join(f'"{h}" TEXT' for h in hdr_clean)
    cur.execute(f'CREATE TABLE staging_wdi_raw ({cols_ddl});')

    placeholders = ','.join('?' for _ in hdr_clean)
    for row in reader:
        if not row:
            continue
        if len(row) != len(hdr_clean):
            print(f"Skipping malformed row: expected {len(hdr_clean)}, got {len(row)}")
            continue
        vals = [None if c in ('', '..') else c for c in row]
        cur.execute(f'INSERT INTO staging_wdi_raw VALUES ({placeholders});', vals)

# 4. Create cleaned lookup tables
cur.execute('DROP TABLE IF EXISTS country_meta;')
cur.execute('''
    CREATE TABLE country_meta AS
    SELECT CountryName AS country_name,
           CountryCode AS country_code
    FROM staging_country_meta;
''')
cur.execute('DROP TABLE IF EXISTS indicator_meta;')
cur.execute('''
    CREATE TABLE indicator_meta AS
    SELECT IndicatorName AS indicator_name,
           IndicatorCode AS indicator_code
    FROM staging_indicator_meta;
''')

# 5. Unpivot into long format using the sanitized headers
years = [h for h in hdr_clean if h.isdigit() or (h.startswith('_') and h[1:].isdigit())]
union_queries = []
for y in years:
    union_queries.append(f"""
        SELECT
            "{hdr_clean[0]}" AS country_name,
            "{hdr_clean[1]}" AS country_code,
            "{hdr_clean[2]}" AS indicator_name,
            "{hdr_clean[3]}" AS indicator_code,
            '{y.lstrip('_')}'        AS year,
            CAST("{y}" AS REAL)    AS value
        FROM staging_wdi_raw
    """
    )
cur.execute('DROP TABLE IF EXISTS gdp_long;')
cur.execute('CREATE TABLE gdp_long AS ' + ' UNION ALL '.join(union_queries))

# Commit and close connection
conn.commit()
conn.close()

print("Database built successfully. 'gdp_long' contains GDP per capita data.")

Database built successfully. 'gdp_long' contains GDP per capita data.


In [27]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# make sure the figures directory is there
os.makedirs('figures', exist_ok=True)

# 1. Connect and load
conn = sqlite3.connect('gdp.db')
gdp = pd.read_sql('SELECT country_code, indicator_code, year, value FROM gdp_long', conn)
conn.close()

# 2. Missing‐value counts by indicator
missing = (
    gdp
    .groupby('indicator_code')['value']
    .apply(lambda x: x.isna().sum())
    .reset_index(name='missing_count')
)
print("\nMissing values by indicator:")
print(missing)

# 3. Descriptive statistics
print("\nDescriptive stats by indicator:")
print(gdp.groupby('indicator_code')['value'].describe())

# 4. Time‐series plots for exemplar countries
for country in ['USA', 'CHN', 'IND']:  
    df_ctry = (
        gdp[gdp['country_code'] == country]
        .pivot_table(index='year', columns='indicator_code', values='value')
        .sort_index()
    )
    plt.figure()
    for ind in df_ctry.columns:
        # convert year strings to int, values to float
        x = df_ctry.index.astype(int)
        y = df_ctry[ind].astype(float)
        plt.plot(x, y, label=ind)
    plt.title(f"Time Series of Indicators for {country}")
    plt.xlabel("Year")
    plt.ylabel("Value")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"figures/{country}_timeseries.png")
    plt.close()

# 5. Correlation matrix
pivot = gdp.pivot_table(
    index=['country_code','year'],
    columns='indicator_code',
    values='value'
)
corr = pivot.corr()
print("\nIndicator correlation matrix:")
print(corr)
corr.to_csv('figures/indicator_correlation.csv')



Missing values by indicator:
   indicator_code  missing_count
0  NY.GDP.PCAP.KD           3282

Descriptive stats by indicator:
                  count          mean           std         min          25%  \
indicator_code                                                                 
NY.GDP.PCAP.KD  14008.0  11205.980261  17910.379144  122.678901  1300.590229   

                        50%           75%            max  
indicator_code                                            
NY.GDP.PCAP.KD  3678.895987  13066.300728  224582.449752  

Indicator correlation matrix:
indicator_code  NY.GDP.PCAP.KD
indicator_code                
NY.GDP.PCAP.KD             1.0


In [29]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import os

# Ensure output directory exists
os.makedirs('figures', exist_ok=True)

# 1. Connect and load GDP per capita data
conn = sqlite3.connect('gdp.db')
df = pd.read_sql(
    "SELECT country_code, year, value FROM gdp_long WHERE indicator_code = 'NY.GDP.PCAP.KD'",
    conn
)
conn.close()

# 2. Descriptive & exploratory analysis
missing_count = df['value'].isna().sum()
print(f"Total missing GDP per capita values: {missing_count}\n")
print("Descriptive statistics for GDP per capita:")
print(df['value'].describe(), "\n")

# Time-series plots for exemplar countries
for country in ['USA', 'CHN', 'IND']:
    df_ts = df[df['country_code'] == country].sort_values('year')
    plt.figure()
    plt.plot(df_ts['year'].astype(int), df_ts['value'].astype(float))
    plt.title(f'GDP per Capita (constant 2015 US$) for {country}')
    plt.xlabel('Year')
    plt.ylabel('GDP per Capita')
    plt.tight_layout()
    plt.savefig(f'figures/{country}_gdp_timeseries.png')
    plt.close()

# 3. Growth rates and trend analysis
df_sorted = df.sort_values(['country_code','year'])
df_sorted['gdp_pct_change'] = df_sorted.groupby('country_code')['value'].pct_change() * 100
growth_stats = (
    df_sorted
    .groupby('country_code')['gdp_pct_change']
    .mean()
    .reset_index(name='avg_pct_change')
    .sort_values('avg_pct_change', ascending=False)
)
growth_stats.to_csv('figures/avg_gdp_growth_by_country.csv', index=False)
print("Average annual % change by country:")
print(growth_stats.head(), "\n")

trend_results = []
for country, grp in df.groupby('country_code'):
    grp = grp.dropna(subset=['value'])
    if len(grp) < 2:
        continue
    X = grp['year'].astype(int).values.reshape(-1, 1)
    y = grp['value'].astype(float).values
    lr = LinearRegression().fit(X, y)
    trend_results.append({
        'country_code': country,
        'slope': lr.coef_[0],
        'intercept': lr.intercept_,
        'r2': lr.score(X, y)
    })
trend_df = pd.DataFrame(trend_results).sort_values('slope', ascending=False)
trend_df.to_csv('figures/gdp_trend_by_country.csv', index=False)
print("Linear trend results by country:")
print(trend_df.head(), "\n")

# 4. Clustering based on 2013-2023 trajectories with relaxed completeness
recent = df[df['year'].astype(int).between(2013, 2023)]
pivot = recent.pivot(index='country_code', columns='year', values='value')

years = list(range(2013, 2024))
min_years = int(0.8 * len(years))  # require 80% data
counts = pivot.notna().sum(axis=1)
pivot_sub = pivot[counts >= min_years].copy()

# Fill remaining gaps
pivot_sub = pivot_sub.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)

if pivot_sub.shape[0] == 0:
    print("No countries meet the completeness threshold for clustering; skipping PCA/KMeans.\n")
else:
    print(f"Clustering on {pivot_sub.shape[0]} countries, each with data for {pivot_sub.shape[1]} years.\n")
    pca = PCA(n_components=2)
    pcs = pca.fit_transform(pivot_sub)
    kmeans = KMeans(n_clusters=3, random_state=42).fit(pcs)

    plt.figure()
    plt.scatter(pcs[:, 0], pcs[:, 1], c=kmeans.labels_)
    plt.title('PCA & KMeans Clusters (2013–2023 GDP per Capita)')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.tight_layout()
    plt.savefig('figures/gdp_pca_kmeans.png')
    plt.close()
    pd.DataFrame({'country_code': pivot_sub.index, 'cluster': kmeans.labels_}) \
      .to_csv('figures/gdp_clusters.csv', index=False)
    print("Clustering results saved.\n")

# 5. Forecasting for USA
usa = df[df['country_code'] == 'USA'].sort_values('year').dropna(subset=['value'])
series = usa.set_index('year')['value'].astype(float)
hw = ExponentialSmoothing(series, trend='add', seasonal=None).fit()
forecast = hw.forecast(5)
forecast_df = forecast.reset_index().rename(columns={'index': 'year', 0: 'forecast'})
forecast_df.to_csv('figures/usa_gdp_forecast.csv', index=False)

plt.figure()
plt.plot(series.index.astype(int), series.values, label='Actual')
plt.plot(forecast.index.astype(int), forecast.values, '--', label='Forecast')
plt.title('USA GDP per Capita: Actual vs Forecast')
plt.xlabel('Year')
plt.ylabel('GDP per Capita')
plt.legend()
plt.tight_layout()
plt.savefig('figures/usa_gdp_forecast.png')
plt.close()
print("USA forecast and plot saved.")



Total missing GDP per capita values: 3282

Descriptive statistics for GDP per capita:
count     14008.000000
mean      11205.980261
std       17910.379144
min         122.678901
25%        1300.590229
50%        3678.895987
75%       13066.300728
max      224582.449752
Name: value, dtype: float64 



/var/folders/x1/f800_tw56gz7v9v_lsh6_sm40000gn/T/ipykernel_12158/4261927175.py:41: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_sorted['gdp_pct_change'] = df_sorted.groupby('country_code')['value'].pct_change() * 100
/var/folders/x1/f800_tw56gz7v9v_lsh6_sm40000gn/T/ipykernel_12158/4261927175.py:82: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pivot_sub = pivot_sub.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)


Average annual % change by country:
    country_code  avg_pct_change
88           GNQ        7.730390
24           BIH        6.787794
40           CHN        6.555799
126          KOR        5.694232
61           EAP        5.491384 

Linear trend results by country:
    country_code        slope     intercept        r2
107          IMN  2260.276165 -4.472009e+06  0.974293
145          MCO  2214.181769 -4.297051e+06  0.895593
141          LUX  1603.514372 -3.123759e+06  0.945071
143          MAC  1303.150890 -2.562297e+06  0.516403
27           BMU  1206.991085 -2.315655e+06  0.816800 

Clustering on 256 countries, each with data for 11 years.

Clustering results saved.

USA forecast and plot saved.


/Users/nicholasrichards/Desktop/anaconda3/envs/qtm151-f24/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
